# Creating Neural Networks on the shoulders of giants that are on the shoulders of other giants

This notebook is a modified version of the one created by @delai50 (https://www.kaggle.com/code/delai50/standing-on-the-head-of-giants-more-fe)
The basis of that notebook is the one created by @maxsarmento (https://www.kaggle.com/code/maxsarmento/lb-0-58978-standing-on-the-shoulder-of-giants/notebook) which in turn is basically the one suggested by @desalegngeb  (https://www.kaggle.com/code/desalegngeb/tps08-logisticregression-and-some-fe/notebook?scriptVersionId=102691691) so credits for them.

In this version I added a function to create a NN using keras and hyperparameter tuning using optuna.
At the end optuna suggests 0 hidden layers, so the model collapses into a perceptron.
Overall the code is more generic to be adaptable also to situations in which it doesn't collapse to a perceptron.

In [1]:
#True-> run optuna parameter search (takes from 10 to 12h on CPU for 800 parameters configuration)
#False-> use params_LR to train an ensemble of NN 
optimizing=False
#How many
if optimizing:
    splits = 5
else:
    splits = 50
n_trials=800

#Parameters for the NN (and optimizer)
params_LR = {
    'n_layers': 0, 
    'dropout': 0.19334570992451758, 
    'reduction': 3, 
    'initial_size': 256, 
    'regularizer_l1': 4.093055122216991e-05, 
    'regularizer_l2': 0.00010688801483338637, 
    'lr':  0.008913875891778679, 
    'batch_size': 128
}
#score :  0.59321

#Configuration of the experiment.
class CFG:
    INPUT_PATH = "/kaggle/input/tabular-playground-series-aug-2022/"
    OUT_PATH = "/kaggle/working/"
    N_SPLITS = splits
    SEED = 2747902
    INFER_MODE = "fold"
    WANDB_PROJECT = 'aug22_tabular'
    
import wandb
from wandb.keras import WandbCallback
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wb_key = user_secrets.get_secret("WANDB_API_KEY")

wandb.login(key=wb_key)

wandb.init(project=CFG.WANDB_PROJECT)


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jgraham20. Use `wandb login --relogin` to force relogin


In [2]:
# IMPORTS
import os
import sys
import joblib

import numpy as np 
import pandas as pd

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, log_loss

!pip install feature_engine
from feature_engine.encoding import WoEEncoder

!git clone https://github.com/analokmaus/kuma_utils.git
sys.path.append("kuma_utils/")
from kuma_utils.preprocessing.imputer import LGBMImputer

!pip install miceforest
import miceforest as mf  

import warnings
warnings.filterwarnings("ignore")

import gc
gc.enable()
import tensorflow as tf
from tensorflow.keras.layers import Input,Dense,Dropout#,BatchNormalization
from tensorflow.keras import regularizers

from sklearn.preprocessing import RobustScaler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 883.4 kB/s eta 0:00:00
Cloning into 'kuma_utils'...
remote: Enumerating objects: 915, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 915 (delta 104), reused 102 (delta 96), pack-reused 795
Receiving objects: 100% (915/915), 679.99 KiB | 1.99 MiB/s, done.
Resolving deltas: 100% (592/592), done.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 406.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 5.5 MB/s eta 0:00:00


In [3]:
def create_model(n_features,initial_size,n_layers,dropout,reduction,regularizer_l1,regularizer_l2,lr,**kwargs):
    inputs=Input(shape=(None,n_features),name="inputs")
    x=inputs
    for i in range(n_layers):
        x=Dense(initial_size//(reduction**i),activation="relu",name=f"dense_{i}",
                kernel_regularizer=regularizers.L1L2(l1=regularizer_l1, l2=regularizer_l2))(x)
        x=Dropout(dropout,name=f"dropout_{i}")(x)
        
    output=Dense(1,activation="sigmoid",name="failure_rate",
                 kernel_regularizer=regularizers.L1L2(l1=regularizer_l1, l2=regularizer_l2))(x)
    model=tf.keras.Model(inputs=inputs,outputs=output)
    
    model.compile(tf.keras.optimizers.Adam(learning_rate=lr),
                  loss="binary_crossentropy",
                  metrics=[tf.keras.metrics.AUC(name="auc")])
    return model

In [4]:
# CREATE FOLDS
def create_folds(df_train):
    cv = StratifiedKFold(n_splits=CFG.N_SPLITS, shuffle=True, random_state=CFG.SEED)
#     cv = GroupKFold()
    df_train["fold"] = -1
    for fold, (_, val_idx) in enumerate(cv.split(df_train, df_train["failure"])):
        df_train.loc[val_idx, "fold"] = fold
    return df_train


In [5]:
# CROSSVALIDATION
def crossvalidation(df_train, df_test, features, params):
    
    scalers=[]
    score_train_list = []
    score_val_list = []
    
    oofs = df_train[["failure", "fold"]].copy()
    oofs["preds"] = None
    models=[]
    for fold in range(CFG.N_SPLITS):

        train_idx = df_train["fold"]!=fold
        val_idx = df_train["fold"]==fold
        scaler = RobustScaler()
        X_train_cv = df_train.loc[train_idx][features]
        X_train_cv=scaler.fit_transform(X_train_cv)
        
        X_val_cv = df_train.loc[val_idx][features]
        X_val_cv=scaler.transform(X_val_cv)
        
        scalers.append(scaler)
        
        y_train_cv = df_train.loc[train_idx]["failure"]
        y_val_cv = df_train.loc[val_idx]["failure"]
        
        model = create_model(len(features),**params)
        
        model.fit(X_train_cv, y_train_cv,
                  validation_data=(X_val_cv,y_val_cv),
                  epochs=100,callbacks=[WandbCallback(), tf.keras.callbacks.EarlyStopping(
                                        monitor="val_auc",
                                        patience=5,
                                        restore_best_weights=True,
                                        mode="max")
                                       ]
                  )
        
        models.append(model)
        
        y_train_cv_pred = model.predict(X_train_cv)
        y_val_cv_pred = model.predict(X_val_cv)
        
        oofs.loc[val_idx, "preds"] = y_val_cv_pred

        score_train = roc_auc_score(y_train_cv, y_train_cv_pred)
        score_val = roc_auc_score(y_val_cv, y_val_cv_pred)
        
        print(f"Train score: {score_train:.7f} / val score: {score_val:.7f} for split {fold}")
        
        score_train_list.append(score_train)
        score_val_list.append(score_val)
    
    oofs.to_csv(os.path.join(CFG.OUT_PATH, f"oofs_seed{CFG.SEED}.csv"))
    
    score_train_avg = np.mean(score_train_list)
    score_val_avg = np.mean(score_val_list)
    score_val_ove = roc_auc_score(oofs["failure"], oofs["preds"])
        
    print(f"Train avg score: {score_train_avg:.7f} / val avg score: {score_val_avg:.7f}")
    print(f"OOFs overall score: {score_val_ove:.7f}")
    return models,scalers

In [6]:
# CROSSVALIDATION
def crossvalidation_optimize(trial,df_train, df_test, features):
    param_grid = {
        "n_layers":trial.suggest_int("n_layers",0,2),
        "dropout":trial.suggest_float("dropout",0.05,0.8),
        "reduction":trial.suggest_int("reduction",1,4),
        "initial_size":trial.suggest_categorical("initial_size", [64,128,256]),
        "regularizer_l1":trial.suggest_float("regularizer_l1",1e-5,0.1,log=True),
        "regularizer_l2":trial.suggest_float("regularizer_l2",1e-5,0.1,log=True),
        "lr":trial.suggest_float("lr",0.0000001,0.1,log=True),
        "batch_size":trial.suggest_categorical("batch_size", [16,32,64,128,256,512,1024,2048]),
    }
    print(param_grid)
    score_val=0
    
    oofs = df_train[["failure", "fold"]].copy()
    oofs["preds"] = None
    models=[]
    for fold in range(CFG.N_SPLITS):

        train_idx = df_train["fold"]!=fold
        val_idx = df_train["fold"]==fold
        
        scaler = RobustScaler()
        
        X_train_cv = df_train.loc[train_idx][features]
        X_train_cv=scaler.fit_transform(X_train_cv)
        
        X_val_cv = df_train.loc[val_idx][features]
        X_val_cv=scaler.transform(X_val_cv)
        
        y_train_cv = df_train.loc[train_idx]["failure"]
        y_val_cv = df_train.loc[val_idx]["failure"]
        
        
        
        
        model = create_model(len(features),**param_grid)

        model.fit(X_train_cv, y_train_cv,
                  validation_data=(X_val_cv,y_val_cv),
                  epochs=100,callbacks=[WandbCallback(), tf.keras.callbacks.EarlyStopping(
                                        monitor="val_auc",
                                        patience=5,
                                        restore_best_weights=True, mode="max"
                                        )
                                       ],
                  verbose=0,
                  batch_size=param_grid["batch_size"]
                  )
        
        models.append(model)
        
        y_val_cv_pred = model.predict(X_val_cv)
        
        oofs.loc[val_idx, "preds"] = y_val_cv_pred

        score_val += roc_auc_score(y_val_cv, y_val_cv_pred)
        
    print("score : ",score_val/CFG.N_SPLITS," ")
    
    return score_val/CFG.N_SPLITS
   

In [7]:
 
# INFERENCE
def inference(df_train, df_test, features, params,models,scalers):
    
    X_train = df_train[features]
    y_train = df_train["failure"]
    X_test = df_test[features]
    y_test_pred = np.zeros(len(X_test))

    
    for fold in range(CFG.N_SPLITS):
        scaler=scalers[fold]
        model = models[fold]
        y_test_pred += model.predict(scaler.transform(X_test)).reshape(-1) / CFG.N_SPLITS
    
    
            
    sub["failure"] = y_test_pred
    sub.to_csv(os.path.join(CFG.OUT_PATH, f"submission.csv"), index=False)
    

In [8]:
# Missing value imputation
imptr = LGBMImputer(n_iter=50)
def impute_nans(df_train, df_test):
    nan_features = [col for col in df_train.columns if df_train[col].isnull().any()]
    for pc in df_train["product_code"].unique():

        df_train.loc[(df_train["product_code"]==pc),nan_features] = \
                    imptr.fit_transform(df_train.loc[(df_train["product_code"]==pc),nan_features])

    nan_features = [col for col in df_test.columns if df_test[col].isnull().any()]
    for pc in df_test["product_code"].unique():

        df_test.loc[(df_test["product_code"]==pc),nan_features] = \
            imptr.fit_transform(df_test.loc[(df_test["product_code"]==pc),nan_features])

    return df_train, df_test

In [9]:


def mice_impute(df_train, df_test):
    
    print(f"Running MICE imputer.")
    
    train_kernel = mf.ImputationKernel(  
        data=df_train, 
        datasets=4, 
        save_all_iterations=True,  
        random_state=CFG.SEED  
    )

    test_kernel = mf.ImputationKernel(  
        data=df_test, 
        datasets=4, 
        save_all_iterations=True,  
        random_state=CFG.SEED  
    )
    train_kernel.mice(3,verbose=True)
    test_kernel.mice(3,verbose=True)

    df_train = train_kernel.complete_data()
    df_test = test_kernel.complete_data()
    
    return df_train, df_test


In [10]:
# PREPROCESSING
def preprocessing(df_train, df_test):
    
    df_test = df_test.drop(['id', 'product_code'], axis=1)
    df_train = df_train.drop(['id', 'product_code'], axis=1)
    
    # Missing indicator
    for df in [df_train, df_test]:
        df["m_3_missing"] = df["measurement_3"].isnull().astype(int)
        df["m_5_missing"] = df["measurement_5"].isnull().astype(int)
    
    # WoE encoding
    woe_encoder = WoEEncoder(variables=["attribute_0"])
    df_train["attribute_0"] = woe_encoder.fit_transform(df_train["attribute_0"].to_frame(), df_train["failure"])
    df_test["attribute_0"] = woe_encoder.transform(df_test["attribute_0"].to_frame())
    
    woe_encoder = WoEEncoder(variables=["attribute_1"])
    df_train["attribute_1"] = woe_encoder.fit_transform(df_train["attribute_1"].to_frame(), df_train["failure"])
    df_test["attribute_1"] = woe_encoder.transform(df_test["attribute_1"].to_frame())
    
    #df_train, df_test = impute_nans(df_train, df_test)
    df_train, df_test = mice_impute(df_train, df_test)
    
    # Area
    for df in [df_train, df_test]:
        df["attribute_2*3"] = df["attribute_2"] * df["attribute_3"]
    
    # Aggregations
    meas_gr1_cols = [f"measurement_{i:d}" for i in list(range(3, 5)) + list(range(9, 17))]
    meas_gr2_cols = [f"measurement_{i:d}" for i in list(range(5, 9))]
    for df in [df_train, df_test]:
        df["meas_gr1_avg"] = np.mean(df[meas_gr1_cols], axis=1)
        df["meas_gr1_std"] = np.std(df[meas_gr1_cols], axis=1)
        df["meas_gr2_avg"] = np.mean(df[meas_gr2_cols], axis=1)
    
    for df in [df_train, df_test]:
        df["meas17/meas_gr2_avg"] = df["measurement_17"] / df["meas_gr2_avg"]
        
    features = ["attribute_0", "measurement_0", "measurement_1", "measurement_2", "m_3_missing", "m_5_missing",
               "meas_gr1_avg", "meas_gr1_std", "attribute_2*3", "loading", "measurement_17", "meas17/meas_gr2_avg"]
    
    return df_train, df_test, features

In [11]:
df_train = pd.read_csv(os.path.join(CFG.INPUT_PATH, "train.csv"))
df_test = pd.read_csv(os.path.join(CFG.INPUT_PATH, "test.csv"))
sub = pd.read_csv(os.path.join(CFG.INPUT_PATH, "sample_submission.csv"))

In [12]:
df_train.columns

Index(['id', 'product_code', 'loading', 'attribute_0', 'attribute_1',
       'attribute_2', 'attribute_3', 'measurement_0', 'measurement_1',
       'measurement_2', 'measurement_3', 'measurement_4', 'measurement_5',
       'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9',
       'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13',
       'measurement_14', 'measurement_15', 'measurement_16', 'measurement_17',
       'failure'],
      dtype='object')

In [13]:
df_train = create_folds(df_train)

In [14]:
df_train, df_test, features = preprocessing(df_train, df_test)

Running MICE imputer.
Initialized logger with name mice 1-3
Dataset 0
1  | loading | measurement_3 | measurement_4 | measurement_5 | measurement_6 | measurement_7 | measurement_8 | measurement_9 | measurement_10 | measurement_11 | measurement_12 | measurement_13 | measurement_14 | measurement_15 | measurement_16 | measurement_17
2  | loading | measurement_3 | measurement_4 | measurement_5 | measurement_6 | measurement_7 | measurement_8 | measurement_9 | measurement_10 | measurement_11 | measurement_12 | measurement_13 | measurement_14 | measurement_15 | measurement_16 | measurement_17
3  | loading | measurement_3 | measurement_4 | measurement_5 | measurement_6 | measurement_7 | measurement_8 | measurement_9 | measurement_10 | measurement_11 | measurement_12 | measurement_13 | measurement_14 | measurement_15 | measurement_16 | measurement_17
Dataset 1
1  | loading | measurement_3 | measurement_4 | measurement_5 | measurement_6 | measurement_7 | measurement_8 | measurement_9 | measuremen

In [15]:
list_str_obj_cols = df_train.columns[df_train.dtypes == "object"].tolist()
list_str_obj_cols

[]

In [16]:
if optimizing:
    import optuna
    from optuna.integration.wandb import WeightsAndBiasesCallback
    
    wandb_kwargs = {"project": CFG.WANDB_PROJECT}
    wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)
    
    study = optuna.create_study(direction="maximize", study_name="NN classifier")
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    func = lambda trial: crossvalidation_optimize(trial, df_train, df_test, features)
    study.optimize(func, n_trials=n_trials, callbacks=[wandbc])
    
    print(f"\tBest value: {study.best_value:.5f}")
    print(f"\tBest params:")

    for key, value in study.best_params.items():
        print(f"\t\t{key}: {value}")

In [17]:
if not optimizing:
    models,scalers=crossvalidation(df_train, df_test, features, params_LR)

2022-08-17 12:23:38.186867: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 12:23:38.195822: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 12:23:38.199949: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 12:23:38.205319: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/100
814/814 [==============================] - 6s 5ms/step - loss: 0.5264 - auc: 0.5602 - val_loss: 0.5089 - val_auc: 0.5944
Epoch 2/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5098 - auc: 0.5851 - val_loss: 0.5131 - val_auc: 0.5829
Epoch 3/100
814/814 [==============================] - 4s 5ms/step - loss: 0.5101 - auc: 0.5857 - val_loss: 0.5108 - val_auc: 0.5904
Epoch 4/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5100 - auc: 0.5869 - val_loss: 0.5117 - val_auc: 0.5918
Epoch 5/100
814/814 [==============================] - 4s 5ms/step - loss: 0.5098 - auc: 0.5862 - val_loss: 0.5086 - val_auc: 0.5987
Epoch 6/100
814/814 [==============================] - 4s 5ms/step - loss: 0.5099 - auc: 0.5860 - val_loss: 0.5086 - val_auc: 0.5887
Epoch 7/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5099 - auc: 0.5853 - val_loss: 0.5108 - val_auc: 0.5938
Epoch 8/100
814/814 [==============================] - 3s 4ms/step - 

2022-08-17 12:25:07.021090: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 12:25:07.022064: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-08-17 12:25:07.022213: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-08-17 12:25:07.022702: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 12:25:07.023608: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 12:25:07.024520: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su

814/814 [==============================] - 5s 5ms/step - loss: 0.5285 - auc: 0.5596 - val_loss: 0.5051 - val_auc: 0.6313
Epoch 2/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5099 - auc: 0.5847 - val_loss: 0.5024 - val_auc: 0.6343
Epoch 3/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5097 - auc: 0.5865 - val_loss: 0.5060 - val_auc: 0.6184
Epoch 4/100
814/814 [==============================] - 4s 5ms/step - loss: 0.5101 - auc: 0.5840 - val_loss: 0.5051 - val_auc: 0.6275
Epoch 5/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5099 - auc: 0.5849 - val_loss: 0.5034 - val_auc: 0.6299
Epoch 6/100
814/814 [==============================] - 4s 5ms/step - loss: 0.5101 - auc: 0.5853 - val_loss: 0.5048 - val_auc: 0.6255
Epoch 7/100
814/814 [==============================] - 4s 5ms/step - loss: 0.5099 - auc: 0.5858 - val_loss: 0.5033 - val_auc: 0.6252
Train score: 0.5887337 / val score: 0.6340423 for split 1


2022-08-17 12:25:35.116823: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 12:25:35.117972: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-08-17 12:25:35.118142: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-08-17 12:25:35.118833: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 12:25:35.120016: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 12:25:35.121235: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su

...

814/814 [==============================] - 6s 7ms/step - loss: 0.5268 - auc: 0.5626 - val_loss: 0.5127 - val_auc: 0.5560
Epoch 2/100
814/814 [==============================] - 4s 5ms/step - loss: 0.5097 - auc: 0.5864 - val_loss: 0.5118 - val_auc: 0.5664
Epoch 3/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5100 - auc: 0.5852 - val_loss: 0.5123 - val_auc: 0.5662
Epoch 4/100
814/814 [==============================] - 4s 5ms/step - loss: 0.5099 - auc: 0.5863 - val_loss: 0.5126 - val_auc: 0.5685
Epoch 5/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5095 - auc: 0.5872 - val_loss: 0.5127 - val_auc: 0.5694
Epoch 6/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5098 - auc: 0.5858 - val_loss: 0.5137 - val_auc: 0.5579
Epoch 7/100
814/814 [==============================] - 4s 4ms/step - loss: 0.5097 - auc: 0.5872 - val_loss: 0.5131 - val_auc: 0.5561
Epoch 8/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5096

2022-08-17 13:03:23.910135: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 13:03:23.911069: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-08-17 13:03:23.911222: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-08-17 13:03:23.911748: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 13:03:23.912704: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 13:03:23.913653: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su

814/814 [==============================] - 4s 4ms/step - loss: 0.5279 - auc: 0.5621 - val_loss: 0.5067 - val_auc: 0.6114
Epoch 2/100
814/814 [==============================] - 4s 5ms/step - loss: 0.5100 - auc: 0.5858 - val_loss: 0.5067 - val_auc: 0.6156
Epoch 3/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5099 - auc: 0.5860 - val_loss: 0.5080 - val_auc: 0.5979
Epoch 4/100
814/814 [==============================] - 4s 5ms/step - loss: 0.5100 - auc: 0.5848 - val_loss: 0.5089 - val_auc: 0.6118
Epoch 5/100
814/814 [==============================] - 4s 5ms/step - loss: 0.5101 - auc: 0.5836 - val_loss: 0.5058 - val_auc: 0.6091
Epoch 6/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5100 - auc: 0.5846 - val_loss: 0.5044 - val_auc: 0.6246
Epoch 7/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5097 - auc: 0.5852 - val_loss: 0.5068 - val_auc: 0.6000
Epoch 8/100
814/814 [==============================] - 5s 6ms/step - loss: 0.5101

2022-08-17 13:04:48.016262: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 13:04:48.017212: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-08-17 13:04:48.017363: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-08-17 13:04:48.017883: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 13:04:48.018913: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 13:04:48.019775: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su

814/814 [==============================] - 7s 7ms/step - loss: 0.5264 - auc: 0.5626 - val_loss: 0.5107 - val_auc: 0.5937
Epoch 2/100
814/814 [==============================] - 4s 4ms/step - loss: 0.5100 - auc: 0.5859 - val_loss: 0.5108 - val_auc: 0.6049
Epoch 3/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5098 - auc: 0.5856 - val_loss: 0.5107 - val_auc: 0.5958
Epoch 4/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5100 - auc: 0.5852 - val_loss: 0.5099 - val_auc: 0.6024
Epoch 5/100
814/814 [==============================] - 4s 4ms/step - loss: 0.5097 - auc: 0.5865 - val_loss: 0.5100 - val_auc: 0.6059
Epoch 6/100
814/814 [==============================] - 4s 4ms/step - loss: 0.5097 - auc: 0.5848 - val_loss: 0.5094 - val_auc: 0.5984
Epoch 7/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5095 - auc: 0.5876 - val_loss: 0.5090 - val_auc: 0.6005
Epoch 8/100
814/814 [==============================] - 3s 4ms/step - loss: 0.5098

In [18]:
if not optimizing:
    inference(df_train, df_test, features, params_LR,models,scalers)

In [19]:
df_train

,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,...,measurement_17,failure,fold,m_3_missing,m_5_missing,attribute_2*3,meas_gr1_avg,meas_gr1_std,meas_gr2_avg,meas17/meas_gr2_avg
0,80.10,0.017894,0.037537,9,5,7,8,4,18.040,12.518,...,764.100,0,20,0,0,45,14.6809,2.127150,16.73350,45.662892
1,84.89,0.017894,0.037537,9,5,14,3,3,18.213,11.540,...,682.057,0,16,0,0,45,15.0001,2.398674,16.56175,41.182665
2,82.43,0.017894,0.037537,9,5,12,1,5,18.057,11.652,...,663.376,0,10,0,0,45,15.8247,2.503762,16.49600,40.214355
3,101.07,0.017894,0.037537,9,5,13,2,6,17.295,11.188,...,826.282,0,48,0,0,45,14.9835,2.657732,17.13950,48.209224
4,188.06,0.017894,0.037537,9,5,9,2,8,19.346,12.950,...,579.885,0,7,0,0,45,15.0582,3.031408,15.53375,37.330651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26565,158.95,0.017894,-0.033831,6,9,6,16,4,16.301,13.259,...,729.131,0,17,0,0,54,14.6996,2.904494,15.94800,45.719275
26566,146.02,0.017894,-0.033831,6,9,10,12,8,17.543,12.351,...,853.924,0,37,0,0,54,14.4502,2.920514,16.94100,50.405761
26567,115.62,0.017894,-0.033831,6,9,1,10,1,15.670,11.535,...,750.364,0,49,0,0,54,14.2516,2.680250,16.51800,45.427049
26568,106.38,0.017894,-0.033831,6,9,2,9,4,18.059,12.853,...,730.156,0,35,0,0,54,15.0357,2.178606,16.52250,44.191617
